In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
path = 'C:/Users/DTLSW/Desktop/Yelp Dataset/yelp_academic_dataset_review.json'
reviews = pd.read_json(path, lines=True, chunksize=10000)

In [7]:
for r in reviews:
    subset = r
    break


In [8]:
subset.shape

(10000, 9)

In [9]:
subset.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [10]:
subset.head(10)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
6,6AxgBCNX_PNTOxmbRSwcKQ,r3zeYsv1XFBRA4dJpL78cw,gmjsEdUsKpj9Xxu6pdjH0g,5,0,2,0,Loved this tour! I grabbed a groupon and the p...,2015-01-03 23:21:18
7,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16
8,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


In [11]:
sortedSubset = subset.sort_values(by=['business_id'])
x = subset['text']

In [12]:
# make list of review text
reviewText = []
for index in enumerate(x):
    reviewText.append(index[1])

In [13]:
print(len(reviewText))

10000


In [14]:
# reviews sorted by business_id
# loop through reviews
# combine the text of the reviews
# remove stop words
# count phrases
# save most common phrases top 100?

# then filter the common phrases and mark each as a positive or negative phrase

# finally loop through the reviews with the phrases list
# group reviews by restaurant
# get most common phrases
# filter out phrases by the list made
# save a list of pros and cons by business_id


In [15]:
import nltk
from nltk.corpus import stopwords

In [16]:
#nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'each', 'other', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'will', 'just', 'don', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'ma', 'mightn', "mightn't", 

Following function is inspired by:
https://dev.to/mattschwartz/quickly-find-common-phrases-in-a-large-list-of-strings-9in

In [17]:
stopwords = stopwords.words('english')

In [18]:
import re

def get_common_phrases(texts, maximum_length=3, minimum_repeat=2) -> dict:
    phrases = {}
    
    for text in texts:
        # clean text
        text = re.sub(r'[.!?,:;/\-\s]',' ', text)
        text = re.sub(r'[\\|@#$&~%\(\)*\"]', '', text)
        
        words = text.split(' ')
        # filter stopwords
        words = [w for w in words if len(w) and w.lower() not in stopwords]
        length = len(words)
        
        # phrases can only be maximum_length words long
        size = length if length <= maximum_length else maximum_length
        while size > 0:
            pos = 0
            # walk over sets of words
            while pos + size <= length:
                phrase = words[pos:pos+size]
                phrase = tuple(w.lower() for w in phrase)
                if phrase in phrases:
                    phrases[phrase] += 1
                else:
                    phrases[phrase] = 1
                pos += 1
            size -= 1
    # remove phrases that are less than the length set by minimum_repeat
    phrases = {k: v for k, v in phrases.items() if v >= minimum_repeat}
    print("Now finding longest phrases")
    longest_phrases = {}
    keys = list(phrases.keys())
    keys.sort(key=len, reverse=True)
    for phrase in keys:
        found = False
        for l_phrase in longest_phrases:
            intersection = set(l_phrase).intersection(phrase)
            if len(intersection) == len(phrase):
                difference = (phrases[phrase] - longest_phrases[l_phrase]) / longest_phrases[l_phrase]
                if difference < 0.25:
                    found = True
                    break
        if not found:
            longest_phrases[phrase] = phrases[phrase]
    
    return longest_phrases
    

## Process for creating a custom filter for relevant phrases

In [15]:
# common_phrases = get_common_phrases(reviewText)

Now finding longest phrases


In [16]:
keys = list(common_phrases.keys())
values = list(common_phrases.values())

#print(keys)

In [17]:
import numpy as np
sorted_value_index = np.argsort(values)
sorted_value_index_desc= sorted_value_index[::-1]
sortedPhrases = {keys[i]: values[i] for i in sorted_value_index_desc}

In [18]:
#print(sortedPhrases)

In [ ]:
outputList = []
for key in sortedPhrases:
    txt = str(key)
    txt = txt.replace("(", "").replace(")","").replace("'","").replace(",","")
    print(txt)
    outputList.append(txt)

In [ ]:
import os.path
save_path = 'C:/Users/DTLSW/Desktop/Yelp Dataset/output'
name_of_file = 'commonPhrases.txt'
complete_path = os.path.join(save_path, name_of_file)
with open(complete_path, 'w') as f:
    for word in outputList:
        f.write(f"{word}\n")

## Process for finding pros and cons

In [180]:
# read filter list
filterFile = 'C:/Users/DTLSW/Desktop\Yelp Dataset/output/combinePhrases.txt'
mappedFile = 'C:/Users/DTLSW/Desktop\Yelp Dataset/output/uniqueProsandCons.txt'
mapPhrases = {}
with open(filterFile) as file:
    for line in file:
        x = line.split(',')
        mapPhrases[x[0]] = x[1].strip('\n')

In [181]:
mapValue = {}
with open (mappedFile) as file:
    for line in file:
        x = line.split(',')
        mapValue[x[0]] = x[1].strip('\n')

In [30]:
#!pip install pypandoc
#!pip install pyspark

  Using cached pypandoc-1.12-py3-none-any.whl (20 kB)
  Using cached pyspark-3.5.0.tar.gz (316.9 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425372 sha256=049e4a18de74aa2e25643c86040edcdfbea75e261b50d3c0ccce71b24469d32d
  Stored in directory: c:\users\dtlsw\appdata\local\pip\cache\wheels\a6\ce\f9\17d82c92f044018df2fe30af63ac043447720d5b2cee39b40f
Successfully built pyspark


In [31]:
#!pip install findspark

In [22]:
from pyspark.sql import SparkSession

In [23]:
# Create Spark Session
spark = SparkSession.builder.master("local").appName("findProsAndCons").getOrCreate()

In [24]:
# group reviews by business_id
reviewKeys = sortedSubset['business_id']
reviewValues = sortedSubset['text']


In [26]:
#pair busines_id with text
reviewPairs = []
for idx, key in enumerate(reviewKeys):
    print(key)
    reviewPairs.append((reviewKeys[idx], reviewValues[idx]))

--ZVrH2X2QXBFdCilbirsw
-02xFuruu85XmDn2xiynJw
-1MhPXk1FglglUAmuPLIGg
-1ueCbvIpUPi8KT95ETTKw
-2Axhv9AZ_n7qjQefECpVw
-2ke_JDOpgTZWqirMFjZcw
-2ke_JDOpgTZWqirMFjZcw
-3AooxIkg38UyUdlz5oXdw
-3AooxIkg38UyUdlz5oXdw
-3dkEoYgH8AlUtBMZvzUfg
-5jrwZnndGs9q3akdkJYJA
-5jrwZnndGs9q3akdkJYJA
-5jrwZnndGs9q3akdkJYJA
-5jrwZnndGs9q3akdkJYJA
-6OjnX3ZdDOhHxWR60wysg
-6OjnX3ZdDOhHxWR60wysg
-7GDqSUaXrpC8Ql7nDBxWA
-7GjicSH_rM8JeZGCXGcUg
-7GjicSH_rM8JeZGCXGcUg
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-ATiAtTikuGuqvaW2O6tNA
-B7E8MFb47FoGcp25lXgxw
-BDSQ7COGF0tcbWJRkTIZw
-BhSR6dAry5-2x3ndjX_9w
-CLLdRWg2JJ6QUVj33HYNg
-DXNvQLhKwunHzg8OjkwXA
-D_L4GHXJQvy2wv0RkrupA
-Fka99c-tJ-epWYNIobqyQ
-Fka99c-tJ-epWYNIobqyQ
-Fka99c-tJ-epWYNIobqyQ
-Fka99c-tJ-epWYNIobqyQ
-HMt-jXxUNBIkO8IUOtE-g
-HMt-jXxUNBIkO8IUOtE-g
-HUDQ5eek6Edz3zuNrE4jw
-HUDQ5eek6Edz3zuNrE4jw
-If0ps0QhOLCYVWQWs9RYg
-If0ps0QhOL

In [28]:
# Distribute the data
reviews_rdd = spark.sparkContext.parallelize(reviewPairs, numSlices=8)


In [29]:
reviews_rdd.take(10)

[('XQfwVwDr-v0ZS3_CbbE5Xw',
  "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker."),
 ('7ATYjTIgM3jUlt4UM3IypQ',
  "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nFor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nThere 

In [30]:
# Group pairs by the key
groupedPairs = reviews_rdd.groupByKey()

In [31]:
groupedPairs = groupedPairs.map(lambda x: (x[0], list(x[1])))

In [32]:
groupedPairs.take(5)

[('gmjsEdUsKpj9Xxu6pdjH0g',
  ["Loved this tour! I grabbed a groupon and the price was great. It was the perfect way to explore New Orleans for someone who'd never been there before and didn't know a lot about the history of the city. Our tour guide had tons of interesting tidbits about the city, and I really enjoyed the experience. Highly recommended tour. I actually thought we were just going to tour through the cemetery, but she took us around the French Quarter for the first hour, and the cemetery for the second half of the tour. You'll meet up in front of a grocery store (seems strange at first, but it's not terribly hard to find, and it'll give you a chance to get some water), and you'll stop at a visitor center part way through the tour for a bathroom break if needed. This tour was one of my favorite parts of my trip!",
   'Very Informative, took us to haunted places (that we were planning to see anyways) and gave us funfilled stories behind each place. Kept us listening all nig

In [188]:
def analyzeBusiness(businessId, texts):
    # restaurant[0] = business_id
    # restaurant[1] = text
    maximum_length=3
    minimum_repeat=2
    
    phrases = {}
    
    for text in texts:
        # clean text
        text = re.sub(r'[.!?,:;/\-\s]',' ', text)
        text = re.sub(r'[\\|@#$&~%\(\)*\"]', '', text)
        
        words = text.split(' ')
        # filter stopwords
        words = [w for w in words if len(w) and w.lower() not in stopwords]
        length = len(words)
        
        # phrases can only be maximum_length words long
        size = length if length <= maximum_length else maximum_length
        while size > 0:
            pos = 0
            # walk over sets of words
            while pos + size <= length:
                phrase = words[pos:pos+size]
                phrase = tuple(w.lower() for w in phrase)
                if phrase in phrases:
                    phrases[phrase] += 1
                else:
                    phrases[phrase] = 1
                pos += 1
            size -= 1
    # remove phrases that are less than the length set by minimum_repeat
    phrases = {k: v for k, v in phrases.items() if v >= minimum_repeat}
    longest_phrases = {}
    keys = list(phrases.keys())
    keys.sort(key=len, reverse=True)
    for phrase in keys:
        found = False
        for l_phrase in longest_phrases:
            intersection = set(l_phrase).intersection(phrase)
            if len(intersection) == len(phrase):
                print(phrases[phrase])
                print(longest_phrases[l_phrase])
                difference = float(phrases[phrase] - len(longest_phrases[l_phrase])) / len(longest_phrases[l_phrase])
                if difference < 0.25:
                    found = True
                    break
        if not found:
            currentPhrase = ' '.join(phrase)
            if currentPhrase in mapPhrases:
                if currentPhrase not in longest_phrases:
                    longest_phrases[mapPhrases[currentPhrase]] = mapValue[mapPhrases[currentPhrase]]
    returnObj = dict()
    returnObj['business_id'] = businessId
    returnObj['pos'] = []
    returnObj['neg'] = []
    for phrase in longest_phrases:
        if longest_phrases[phrase] == 'pos':
            returnObj['pos'].append(phrase)
        else:
            returnObj['neg'].append(phrase)
    return returnObj

In [134]:
example_phrases = analyzeBusiness(example[4][0],example[4][1])

Now finding longest phrases


In [135]:
print(example_phrases)

{'business_id': 'ltBBYdNzkeKdCNPDAsxwAA', 'pos': ['great ambiance', 'great drinks', 'great service', 'great food', 'friendly staff', 'great atmosphere'], 'neg': ['long wait times']}


In [145]:
findProsAndCons = groupedPairs.map(lambda x: analyzeBusiness(x[0],x[1]))

# final output
# {
#    business_id: string
#    pros: []
#    cons: []
# }

In [154]:
findProsAndCons.take(5)

[{'business_id': 'gmjsEdUsKpj9Xxu6pdjH0g',
  'pos': ['helpful staff'],
  'neg': []},
 {'business_id': 'EQ-TZ2eeD_E0BHuvoaeG5Q',
  'pos': ['great service', 'friendly staff', 'great food'],
  'neg': ['long wait times', 'inconsistent']},
 {'business_id': 'vC2qm1y3Au5czBtbhc-DNw', 'pos': [], 'neg': []},
 {'business_id': 'MWmXGQ98KbRo3vsS5nZhMA', 'pos': [], 'neg': []},
 {'business_id': 'ltBBYdNzkeKdCNPDAsxwAA',
  'pos': ['great ambiance',
   'great drinks',
   'great service',
   'great food',
   'friendly staff',
   'great atmosphere'],
  'neg': ['long wait times']}]

In [189]:
examples = groupedPairs.take(10000)
finalOutput = []
for idx, example in enumerate(examples):
    print(idx)
    analyzeBusiness(example[0],example[1])
    finalOutput.append(analyzeBusiness(example[0],example[1]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
2
pos
2
pos
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
2
pos
2
pos
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
2
pos
2
pos
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
26

1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126


3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833


In [166]:
print(examples[13][0])
print(examples[13][1])

oQ5CPRt0R3AzFvcjNOqB1w
["Honestly the food doesn't knock my socks off but other people seem to love this place. I go because my husband likes it as for me I'd rather go to a different BBQ spot. I guess it also depends on what you order.", "The Tuck Special. So good I would drive all the way back out to Nashville from TX just to have it again. It's a mix of strange ingredients- brisket, egg, pimento cheese but combined they are an amazing flavor combination. My husband didn't care for the BBQ sauce on the table, but meat was pretty good. I can see how this place gets crowded, but if you have patience and look up the menu ahead of time, it should be manageable.", 'I ordered the Pulled Pork Nachos.  They were delicious!  Instead of tortilla chips they are served with potato chips...perfect to share with a Beer or Bloody Mary', 'First time visit the other day, amazing!!!! Best hot chicken anywhere!!! Will def be a must stop every time we come to Nashville!!!', "We stopped here on our way t

In [174]:
analyzeBusiness(examples[13][0],examples[13][1])

Now finding longest phrases
2
pos


{'business_id': 'oQ5CPRt0R3AzFvcjNOqB1w',
 'pos': ['great food', 'great location', 'great service', 'friendly staff'],
 'neg': ['feels cramped', 'long wait times', 'inconsistent']}

In [190]:
print(finalOutput[0])

{'business_id': 'gmjsEdUsKpj9Xxu6pdjH0g', 'pos': ['helpful staff'], 'neg': []}


In [191]:
import json

In [192]:
output_save_path = 'C:/Users/DTLSW/Desktop/Yelp Dataset/output/finalOutput.json'
with open(output_save_path, 'w') as f:
    json.dump(finalOutput, f)

In [198]:
output_save_path = 'C:/Users/DTLSW/Desktop/Yelp Dataset/output/reviewID.txt'
with open(output_save_path, 'w') as f:
    for word in reviewKeys:
        f.write('"' + f"{word}" + '",')